<a href="https://colab.research.google.com/github/ymoslem/LLMs/blob/main/inference/LLama2-CTranslate2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLama 2 with CTranslate2

1. To use Llama2, you must first request access [here](https://huggingface.co/meta-llama/Llama-2-70b-chat-hf).
2. You will also have to agree to the license [here](https://ai.meta.com/resources/models-and-libraries/llama-downloads/).
3. When you receive the email from step #2, follow the instructions to download "tokenizer.model" too if you want to use SentencePiece directly; otherwise, you can use Hugging Face's `LlamaTokenizerFast`
4. Finally, obtain a Hugging Face access token [here](https://huggingface.co/settings/tokens).

In [18]:
!pip3 install accelerate ctranslate2 sentencepiece transformers einops &> /dev/null

In [ ]:
# Log in to huggingface - change the access token

!huggingface-cli login --token "hf_your_access_token"

In [ ]:
# [Recommended] Save your files in Google Drive

# !mkdir -p /content/drive/MyDrive/models/ct2-meta-llama
# %cd /content/drive/MyDrive/models/ct2-meta-llama

In [ ]:
# Convert LLama to the CTranslate2 format (2mins for Llama-2-7b-chat-hf)
# Model names: meta-llama/Llama-2-{7,13,70}b-chat-hf

!ct2-transformers-converter --model meta-llama/Llama-2-13b-chat-hf --quantization int8 --output_dir ct2-llama-2-13b-chat-hf --trust_remote_code --low_cpu_mem_usage


In [ ]:
# You can Convert Falcon in the same way

# !ct2-transformers-converter --model tiiuae/falcon-7b-instruct --quantization int8 --output_dir ct2-falcon-7b-instruct --trust_remote_code

In [26]:
# Load the model

import ctranslate2
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# The path of the model you just converted to the CTranslate2
model_path = "ct2-llama-2-13b-chat-hf"

generator = ctranslate2.Generator(model_path, device=device)

In [61]:
# Load the tokenizer

from transformers import LlamaTokenizerFast

model_name = "meta-llama/Llama-2-13b-chat-hf"

tokenizer = LlamaTokenizerFast.from_pretrained(model_name,
                                              padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# [Optional] save the tokenizer to use later
# tokenizer.save_pretrained("./llama2_tokenizer/")

## Example 1: Translation

In [62]:
# Prompt 1

src = "Spanish"
tgt = "English"

sentences = ["Tom tiene dos hermanos que viven en Boston.",
            "La isla está deshabitada.",
            "La escuela se encuentra en la colina."]

query = f"Translate the following text from {src} to {tgt}:"

prompts = [f"{query}\n{src}: {sent}\n{tgt}:" for sent in sentences]

print(*prompts, sep="\n\n")

Translate the following text from Spanish to English:
Spanish: Tom tiene dos hermanos que viven en Boston.
English:

Translate the following text from Spanish to English:
Spanish: La isla está deshabitada.
English:

Translate the following text from Spanish to English:
Spanish: La escuela se encuentra en la colina.
English:


In [84]:
# Prompt 2

src = "Spanish"
tgt = "English"

sentences = ["Tom tiene dos hermanos que viven en Boston.",
            "La isla está deshabitada.",
            "La escuela se encuentra en la colina."]

query = f"Translate the following text from {src} to {tgt}:"

prompts = [f"{query}\n{sent}\n{tgt}:" for sent in sentences]

print(*prompts, sep="\n\n")

Translate the following text from Spanish to English:
Tom tiene dos hermanos que viven en Boston.
English:

Translate the following text from Spanish to English:
La isla está deshabitada.
English:

Translate the following text from Spanish to English:
La escuela se encuentra en la colina.
English:


In [85]:
# Tokenize the prompts

import torch

tokens = [tokenizer.convert_ids_to_tokens(tokenizer.encode(prompt)) for prompt in prompts]

In [86]:
print(*tokens, sep="\n")

['<s>', '▁Trans', 'late', '▁the', '▁following', '▁text', '▁from', '▁Spanish', '▁to', '▁English', ':', '<0x0A>', 'Tom', '▁tiene', '▁dos', '▁h', 'erman', 'os', '▁que', '▁v', 'iven', '▁en', '▁Boston', '.', '<0x0A>', 'English', ':']
['<s>', '▁Trans', 'late', '▁the', '▁following', '▁text', '▁from', '▁Spanish', '▁to', '▁English', ':', '<0x0A>', 'La', '▁is', 'la', '▁está', '▁des', 'hab', 'it', 'ada', '.', '<0x0A>', 'English', ':']
['<s>', '▁Trans', 'late', '▁the', '▁following', '▁text', '▁from', '▁Spanish', '▁to', '▁English', ':', '<0x0A>', 'La', '▁esc', 'uela', '▁se', '▁encuentra', '▁en', '▁la', '▁col', 'ina', '.', '<0x0A>', 'English', ':']


In [87]:
# Generate outputs

results = generator.generate_batch(tokens,
                                   sampling_topk=10,
                                   max_length=20,
                                   include_prompt_in_result=False,
                                   min_length=3,
                                  )
outputs = [tokenizer.decode(result.sequences_ids[0]).strip() for result in results]

# Remove over-generation
clean_outputs = [output.split("\n")[0] for output in outputs]

print(*clean_outputs, sep="\n\n")

Tom has two brothers who live in Boston.

The island is uninhabited.

The school is located on the hill.


## Example 2: Summarization

In [122]:
query = f"Summarize the following paper:"

abstracts = [
    """Deep neural nets with a large number of parameters are very powerful machine learning systems. However, overfitting is a serious problem in such networks. Large networks are also slow to use, making it difficult to deal with overfitting by combining the predictions of many different large neural nets at test time. Dropout is a technique for addressing this problem. The key idea is to randomly drop units (along with their connections) from the neural network during training. This prevents units from co-adapting too much. During training, dropout samples from an exponential number of different "thinned" networks. At test time, it is easy to approximate the effect of averaging the predictions of all these thinned networks by simply using a single unthinned network that has smaller weights. This significantly reduces overfitting and gives major improvements over other regularization methods. We show that dropout improves the performance of neural networks on supervised learning tasks in vision, speech recognition, document classification and computational biology, obtaining state-of-the-art results on many benchmark data sets.""",
    """Consistency is a key requirement of high-quality translation. It is especially important to adhere to pre-approved terminology and adapt to corrected translations in domain-specific projects. Machine translation (MT) has achieved significant progress in the area of domain adaptation. However, real-time adaptation remains challenging. Large-scale language models (LLMs) have recently shown interesting capabilities of in-context learning, where they learn to replicate certain input-output text generation patterns, without further fine-tuning. By feeding an LLM at inference time with a prompt that consists of a list of translation pairs, it can then simulate the domain and style characteristics. This work aims to investigate how we can utilize in-context learning to improve real-time adaptive MT. Our extensive experiments show promising results at translation time. For example, LLMs can adapt to a set of in-domain sentence pairs and/or terminology while translating a new sentence. We observe that the translation quality with few-shot in-context learning can surpass that of strong encoder-decoder MT systems, especially for high-resource languages. Moreover, we investigate whether we can combine MT from strong encoder-decoder models with fuzzy matches, which can further improve translation quality, especially for less supported languages. We conduct our experiments across five diverse language pairs, namely English-to-Arabic (EN-AR), English-to-Chinese (EN-ZH), English-to-French (EN-FR), English-to-Kinyarwanda (EN-RW), and English-to-Spanish (EN-ES)."""
    ]

authors = ["Srivastava at al. (2014)",
           "Moslem et al. (2023)"]

prompts = [f"{query}\nAbstract: {abstract}\nSummary: {author}" for abstract, author in zip(abstracts, authors)]

print(*prompts, sep="\n\n")

Summarize the following paper:
Abstract: Deep neural nets with a large number of parameters are very powerful machine learning systems. However, overfitting is a serious problem in such networks. Large networks are also slow to use, making it difficult to deal with overfitting by combining the predictions of many different large neural nets at test time. Dropout is a technique for addressing this problem. The key idea is to randomly drop units (along with their connections) from the neural network during training. This prevents units from co-adapting too much. During training, dropout samples from an exponential number of different "thinned" networks. At test time, it is easy to approximate the effect of averaging the predictions of all these thinned networks by simply using a single unthinned network that has smaller weights. This significantly reduces overfitting and gives major improvements over other regularization methods. We show that dropout improves the performance of neural ne

In [135]:
# Tokenize the prompts

import torch

tokens = [tokenizer.convert_ids_to_tokens(tokenizer.encode(prompt)) for prompt in prompts]

In [136]:
print(tokens[0])

['<s>', '▁Sum', 'mar', 'ize', '▁the', '▁following', '▁paper', ':', '<0x0A>', 'Abstract', ':', '▁Deep', '▁neural', '▁n', 'ets', '▁with', '▁a', '▁large', '▁number', '▁of', '▁parameters', '▁are', '▁very', '▁powerful', '▁machine', '▁learning', '▁systems', '.', '▁However', ',', '▁over', 'f', 'itting', '▁is', '▁a', '▁serious', '▁problem', '▁in', '▁such', '▁networks', '.', '▁Lar', 'ge', '▁networks', '▁are', '▁also', '▁slow', '▁to', '▁use', ',', '▁making', '▁it', '▁difficult', '▁to', '▁deal', '▁with', '▁over', 'f', 'itting', '▁by', '▁combining', '▁the', '▁predictions', '▁of', '▁many', '▁different', '▁large', '▁neural', '▁n', 'ets', '▁at', '▁test', '▁time', '.', '▁Drop', 'out', '▁is', '▁a', '▁technique', '▁for', '▁address', 'ing', '▁this', '▁problem', '.', '▁The', '▁key', '▁idea', '▁is', '▁to', '▁randomly', '▁drop', '▁units', '▁(', 'al', 'ong', '▁with', '▁their', '▁connections', ')', '▁from', '▁the', '▁neural', '▁network', '▁during', '▁training', '.', '▁This', '▁prevents', '▁units', '▁from', '▁

In [110]:
# Generate outputs

results = generator.generate_batch(tokens,
                                   sampling_topk=15,
                                   max_length=200,
                                   include_prompt_in_result=False,
                                   min_length=50,
                                  )
outputs = [tokenizer.decode(result.sequences_ids[0]).strip() for result in results]
final_outputs = [f"{author} {output}" for author, output in zip(authors, outputs)]

print(*final_outputs, sep="\n\n")

Srivastava at al. (2014) propose a technique called "dropout" to prevent overfitting in deep neural networks. The key idea is to randomly drop units and their connections during training, preventing co-adaptation and reducing overfitting. The authors show that dropout improves performance on various tasks in computer vision, speech recognition, document classification, and bioinformatics, achieving state-of-the-art results on many benchmark datasets. The paper provides a simple and efficient way to reduce overfitting in deep neural networks, and has had a significant impact on the field of machine learning.

Moslem et al. (2023) explore the use of in-context learning in large language models (LLMs) to improve real-time adaptive machine translation (MT). The authors investigate whether LLMs can adapt to a set of in-domain sentence pairs and/or terminology while translating a new sentence. The results show that the translation quality with few-shot in-context learning can surpass that of

## Example 3: Using SentencePiece directly

In [69]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor("tokenizer.model")

sp.encode_as_pieces("This is a test")

['▁This', '▁is', '▁a', '▁test']

In [144]:
tokens = sp.encode_as_pieces(prompts)
tokens = [['<s>'] + prompt for prompt in tokens]

In [145]:
print(tokens[0])

['<s>', '▁Sum', 'mar', 'ize', '▁the', '▁following', '▁paper', ':', '<0x0A>', 'Abstract', ':', '▁Deep', '▁neural', '▁n', 'ets', '▁with', '▁a', '▁large', '▁number', '▁of', '▁parameters', '▁are', '▁very', '▁powerful', '▁machine', '▁learning', '▁systems', '.', '▁However', ',', '▁over', 'f', 'itting', '▁is', '▁a', '▁serious', '▁problem', '▁in', '▁such', '▁networks', '.', '▁Lar', 'ge', '▁networks', '▁are', '▁also', '▁slow', '▁to', '▁use', ',', '▁making', '▁it', '▁difficult', '▁to', '▁deal', '▁with', '▁over', 'f', 'itting', '▁by', '▁combining', '▁the', '▁predictions', '▁of', '▁many', '▁different', '▁large', '▁neural', '▁n', 'ets', '▁at', '▁test', '▁time', '.', '▁Drop', 'out', '▁is', '▁a', '▁technique', '▁for', '▁address', 'ing', '▁this', '▁problem', '.', '▁The', '▁key', '▁idea', '▁is', '▁to', '▁randomly', '▁drop', '▁units', '▁(', 'al', 'ong', '▁with', '▁their', '▁connections', ')', '▁from', '▁the', '▁neural', '▁network', '▁during', '▁training', '.', '▁This', '▁prevents', '▁units', '▁from', '▁

In [146]:
# Generate outputs

results = generator.generate_batch(tokens,
                                   sampling_topk=15,
                                   max_length=200,
                                   include_prompt_in_result=False,
                                   min_length=50,
                                  )
outputs = [tokenizer.decode(result.sequences_ids[0]).strip() for result in results]
final_outputs = [f"{author} {output}" for author, output in zip(authors, outputs)]

print(*final_outputs, sep="\n\n")

Srivastava at al. (2014) introduce dropout as a regularization technique to address overfitting in deep neural networks. The method randomly drops units and their connections during training, preventing co-adaptation and improving generalization. By training a single unthinned network at test time, dropout achieves better results on various supervised learning tasks compared to other regularization methods, leading to state-of-the-art performance on benchmark datasets in vision, speech recognition, document classification, and computational biology.

Moslem et al. (2023) investigate the use of in-context learning in large-scale language models (LLMs) to improve real-time adaptive machine translation (MT). The authors feed an LLM with a prompt consisting of a list of translation pairs and observe promising results in adapting to new sentences and terminology in different domains and languages. The study shows that few-shot in-context learning can improve translation quality, especially 